In [193]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [108]:
# Import aircraft info
aircraft_info = pd.read_excel('Aircraft_fleet_info.xlsx')

# Transpose the dataframe 
aircraft_info = aircraft_info.T

# Promote the first row to column headers and drop the first row
aircraft_info.reset_index(inplace=True)
aircraft_info.columns = aircraft_info.iloc[0]
aircraft_info = aircraft_info.drop(aircraft_info.index[0])

# Split the Aircraft Type column into two columns at : and drop the original column and rename the new columns
aircraft_info[['Aircraft Type', 'Aircraft Type Name']] = aircraft_info['Aircraft Type'].str.split(':', expand=True)
aircraft_info.set_index('Aircraft Type', inplace=True)

# Make it a dictionary for each aircraft type
aircraft_info = aircraft_info.to_dict('index')

# Add kerosen flag to the dictionary
aircraft_info['Type 1']['Electric'] = 0
aircraft_info['Type 2']['Electric'] = 1
aircraft_info['Type 3']['Electric'] = 1

# Make a list of aircraft types
aircraft_types = list(aircraft_info.keys())


In [109]:
# Import distances 
distances = pd.read_csv('Group_4Regional_Distances.csv')

# Rename the unammed column to 'Origin'
distances.rename(columns={'Unnamed: 0':'Origin'}, inplace=True)
distances.set_index('Origin', inplace=True)

# Make it a dictionary
distances = distances.to_dict()

In [129]:
# Import airport info 
airport_info = pd.read_csv('Group_4Regional_Airport_info.csv')

hub = airport_info['ICAO Code'].iloc[0]

# Make it a dictionary using the ICAO code as the key and the rest of the info in a sub-dictionary
airport_info = airport_info.set_index('ICAO Code').T.to_dict('dict')

# Create a list of the airport ICAO codes
airport_list = list(airport_info.keys())

In [178]:
# Import demand data
daily_demand = pd.read_csv('Group_4Regional_Demand.csv')
daily_demand.rename(columns={'Unnamed: 0':'Origin'}, inplace=True)
daily_demand.set_index('Origin', inplace=True)
daily_demand = daily_demand.to_dict('index')

{'LIRF': {'LIRF': 0,
  'LIPZ': 680,
  'LICJ': 390,
  'LIRQ': 750,
  'LIEE': 275,
  'LIRN': 810,
  'LIPX': 595,
  'LIBP': 410,
  'LIEO': 290,
  'LIMJ': 415,
  'LIPY': 415,
  'LIPQ': 440},
 'LIPZ': {'LIRF': 680,
  'LIPZ': 0,
  'LICJ': 320,
  'LIRQ': 720,
  'LIEE': 260,
  'LIRN': 525,
  'LIPX': 845,
  'LIBP': 375,
  'LIEO': 240,
  'LIMJ': 510,
  'LIPY': 495,
  'LIPQ': 620},
 'LICJ': {'LIRF': 395,
  'LIPZ': 305,
  'LICJ': 0,
  'LIRQ': 310,
  'LIEE': 175,
  'LIRN': 395,
  'LIPX': 305,
  'LIBP': 220,
  'LIEO': 150,
  'LIMJ': 240,
  'LIPY': 210,
  'LIPQ': 230},
 'LIRQ': {'LIRF': 680,
  'LIPZ': 700,
  'LICJ': 335,
  'LIRQ': 0,
  'LIEE': 230,
  'LIRN': 590,
  'LIPX': 790,
  'LIBP': 380,
  'LIEO': 250,
  'LIMJ': 495,
  'LIPY': 450,
  'LIPQ': 470},
 'LIEE': {'LIRF': 325,
  'LIPZ': 230,
  'LICJ': 180,
  'LIRQ': 270,
  'LIEE': 0,
  'LIRN': 245,
  'LIPX': 230,
  'LIBP': 140,
  'LIEO': 110,
  'LIMJ': 190,
  'LIPY': 145,
  'LIPQ': 160},
 'LIRN': {'LIRF': 800,
  'LIPZ': 590,
  'LICJ': 465,
  'LIRQ': 50

In [181]:

# Import the houly coefficients
hourly_coefficients = pd.read_csv('Group_4Hourly_coefficients.csv')
hourly_coefficients.rename(columns={'Unnamed: 0':'id', 'Unnamed: 1':'Origin Name', 'Unnamed: 2':'Origin ICAO'}, inplace=True)
hourly_coefficients.drop(columns=['id','Unnamed: 0.1','Origin Name'], inplace=True)
hourly_coefficients.set_index('Origin ICAO', inplace=True)

# Compute the hourly demand for each OD pair
hour_demand = {}
for origin in airport_list:
    for destination in airport_list:
        od_demand = daily_demand[origin][destination]
        hour_demand[origin,destination] = hourly_coefficients  * od_demand 


In [184]:
len(list(hour_demand.keys()))

144

In [111]:
# Revenue

''' Create Yield matrix dict from formula in RPK using distance matrix
    Formula: Yield = 5.9 ∙ dij^(−0.76) + 0.043 '''

yield_matrix = {}
for i in airport_list:
    yield_matrix[i] = {}
    for j in airport_list:
        if i == j:
            yield_matrix[i][j] = 0
        else:
            yield_matrix[i][j] = 5.9 * (distances[i][j] ** (-0.76)) + 0.043


In [140]:
# Costs 
# Operational costs

''' Fixed costs = CXk  '''
fixed = {}
for k in aircraft_types:
    fixed[k] = aircraft_info[k]['Fixed Operating Cost (Per Fligth Leg)  [€]']

''' Time based costs i -> j
    Formula: CTkij = cTk ∙ dij / Vk '''
time_based = {}
for k in aircraft_types:
    time_based[k] = {}
    for i in airport_list:
        time_based[k][i] = {}
        for j in airport_list:
            time_based[k][i][j] = aircraft_info[k]['Cost per Hour'] * distances[i][j] / aircraft_info[k]['Speed [km/h]']

''' Fuel costs i -> j
    Formula: CFkij = cFk ∙ dij ∙ f / 1.5'''
fuel_gal_cost = 1.42 
fuel = {}
for k in aircraft_types:
    fuel[k] = {}
    for i in airport_list:
        fuel[k][i] = {}
        for j in airport_list:
            if aircraft_info[k]['Electric'] == 1:
                fuel[k][i][j] = 0
            else:
                fuel[k][i][j] = aircraft_info[k]['Fuel Cost Parameter'] * distances[i][j] * fuel_gal_cost / 1.5

''' Energy costs i -> j
    Formula: CEkij = e ∙ Gk ∙ dij / Rk '''
energy = {}
for k in aircraft_types:
    energy[k] = {}
    for i in airport_list:
        energy[k][i] = {}
        for j in airport_list:
            if aircraft_info[k]['Electric'] == 0:
                energy[k][i][j] = 0
            else:
                energy[k][i][j] = 0.07 * aircraft_info[k]['Batteries energy (kWh)'] * distances[i][j] / aircraft_info[k]['Maximum Range [km]']

''' Total costs i -> j
    Formula: CXk + CTkij + CFkij + CEkij '''
hub_factor = 0.7
op_costs = {}
for k in aircraft_types:
    op_costs[k] = {}
    for i in airport_list:
        op_costs[k][i] = {}
        for j in airport_list:
            if i == hub or j == hub:
                op_costs[k][i][j] = (fixed[k] + time_based[k][i][j] + fuel[k][i][j]) * hub_factor + energy[k][i][j]
            else:
                op_costs[k][i][j] = fixed[k] + time_based[k][i][j] + fuel[k][i][j] + energy[k][i][j]
            if i == j:
                op_costs[k][i][j] = 0
                

In [189]:
''' Time i -> j
    Formula: dij / Vk + TATk + 30 (landing and takeoff) + additional charging time (if electric) '''
times = {}
for k in aircraft_types:
    times[k] = {}
    for i in airport_list:
        times[k][i] = {}
        for j in airport_list:
            times[k][i][j] = {}
            times[k][i][j] = (distances[i][j] / aircraft_info[k]['Speed [km/h]'] * 60
                              + aircraft_info[k]['Average TAT [min]']
                              + 30
                              + (aircraft_info[k]['Additional charging time'] if aircraft_info[k]['Electric'] == 1 else 0) if i != j else 0) / 60

In [161]:
# Create a list of all possible time steps
time_steps = list(reversed(np.arange(0, 24.1, 0.1)))

In [194]:
first_fligth = {}
ac = aircraft_info['Type 1']
for i in airport_list:
    first_fligth[i] = {}
    for t in time_steps:
        first_fligth[i]['Time'] = math.floor((24 - times['Type 1'][i][hub]) * 10) / 10

first_fligth

{'LIRF': {'Time': 24.0},
 'LIPZ': {'Time': 22.3},
 'LICJ': {'Time': 22.3},
 'LIRQ': {'Time': 22.6},
 'LIEE': {'Time': 22.3},
 'LIRN': {'Time': 22.7},
 'LIPX': {'Time': 22.3},
 'LIBP': {'Time': 22.7},
 'LIEO': {'Time': 22.6},
 'LIMJ': {'Time': 22.3},
 'LIPY': {'Time': 22.6},
 'LIPQ': {'Time': 22.2}}

In [201]:
non_doable_origins = {}
for k in aircraft_types:
    non_doable_origins[k] = {}
    for i in airport_list:
        if distances[i][hub] <= aircraft_info[k]['Maximum Range [km]'] and aircraft_info[k]['Runway Required [m]'] < airport_info[i]['Runway (m)']:
            non_doable_origins[k][i] = 1

In [202]:
non_doable_origins

{'Type 1': {'LIRF': 1,
  'LIPZ': 1,
  'LICJ': 1,
  'LIRQ': 1,
  'LIEE': 1,
  'LIRN': 1,
  'LIPX': 1,
  'LIBP': 1,
  'LIEO': 1,
  'LIMJ': 1,
  'LIPY': 1,
  'LIPQ': 1},
 'Type 2': {'LIRF': 1,
  'LIPZ': 1,
  'LICJ': 1,
  'LIRQ': 1,
  'LIEE': 1,
  'LIRN': 1,
  'LIPX': 1,
  'LIBP': 1,
  'LIEO': 1,
  'LIMJ': 1,
  'LIPY': 1,
  'LIPQ': 1},
 'Type 3': {'LIRF': 1,
  'LIRQ': 1,
  'LIEE': 1,
  'LIRN': 1,
  'LIBP': 1,
  'LIEO': 1,
  'LIMJ': 1,
  'LIPY': 1}}

In [195]:
aircraft_info

{'Type 1': {'Speed [km/h]': 550.0,
  'Seats': 45.0,
  'Average TAT [min]': 25.0,
  'Additional charging time': nan,
  'Maximum Range [km]': 1500.0,
  'Runway Required [m]': 1400.0,
  'Lease Cost [€]': 2142.8571428571427,
  'Fixed Operating Cost (Per Fligth Leg)  [€]': 300.0,
  'Cost per Hour': 750.0,
  'Fuel Cost Parameter': 1.0,
  'Batteries energy (kWh)': nan,
  'Fleet': 5.0,
  'Aircraft Type Name': ' Regional turboprop',
  'Electric': 0},
 'Type 2': {'Speed [km/h]': 480.0,
  'Seats': 48.0,
  'Average TAT [min]': 25.0,
  'Additional charging time': 45.0,
  'Maximum Range [km]': 1000.0,
  'Runway Required [m]': 950.0,
  'Lease Cost [€]': 3142.0,
  'Fixed Operating Cost (Per Fligth Leg)  [€]': 120.0,
  'Cost per Hour': 750.0,
  'Fuel Cost Parameter': nan,
  'Batteries energy (kWh)': 8216.0,
  'Fleet': 3.0,
  'Aircraft Type Name': ' Electric regional aircraft ',
  'Electric': 1},
 'Type 3': {'Speed [km/h]': 350.0,
  'Seats': 20.0,
  'Average TAT [min]': 20.0,
  'Additional charging time

In [196]:
airport_info

{'LIRF': {'City Name': 'Fiumicino',
  'Latitude (deg)': 41.77354,
  'Longitude (deg)': 12.239712,
  'Runway (m)': 3900,
  'Population': 5671181,
  'GDP': 200.84},
 'LIPZ': {'City Name': 'Venezia',
  'Latitude (deg)': 45.5032,
  'Longitude (deg)': 12.3512,
  'Runway (m)': 3300,
  'Population': 5658163,
  'GDP': 164.86},
 'LICJ': {'City Name': 'Palermo',
  'Latitude (deg)': 38.1824,
  'Longitude (deg)': 13.100582,
  'Runway (m)': 3326,
  'Population': 1880634,
  'GDP': 89.36},
 'LIRQ': {'City Name': 'Firenze',
  'Latitude (deg)': 43.808653,
  'Longitude (deg)': 11.201225,
  'Runway (m)': 1750,
  'Population': 5827084,
  'GDP': 118.72},
 'LIEE': {'City Name': 'Cagliari',
  'Latitude (deg)': 39.2515388,
  'Longitude (deg)': 9.0556723,
  'Runway (m)': 2803,
  'Population': 994409,
  'GDP': 35.25},
 'LIRN': {'City Name': 'Napoli',
  'Latitude (deg)': 40.884,
  'Longitude (deg)': 14.2878,
  'Runway (m)': 2012,
  'Population': 6282927,
  'GDP': 109.63},
 'LIPX': {'City Name': 'Verona',
  'Lati

In [ ]:
# Lets make an example for Type 1 aircraft
# Possible states at each time step starting at 24.0 from the hub
# Create a dictionary for each time step
states = {}

ac = aircraft_info['Type 1']
for t in time_steps:
    states[t] = {}
    for i in airport_list:
        if org_dest[i][hub]['Time'] < 

